# Cleaning File
Get the basic company information here.

In [8]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [9]:
file = "../../data/companyData/compustat_2010s.csv"
compustat = pd.read_csv(file, encoding = 'unicode_escape')

In [10]:
compustat.fyearq.value_counts()

2013    47496
2012    47264
2014    46972
2015    46272
2016    45754
2019    45631
2018    45470
2017    45449
2011    44585
2010    41695
2020    22861
2009     1512
Name: fyearq, dtype: int64

In [11]:
compustat.head()

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,datafqtr,chq,cogsq,gdwlq,invtq,invwipq,niq,revtq,costat,naics
0,1004,20100228,2009,3.0,INDL,C,D,STD,AIR,000361105,...,2009Q3,117.526,242.629,NaN,444.645,NaN,9.912,300.845,A,423860.0
1,1004,20100531,2009,4.0,INDL,C,D,STD,AIR,000361105,...,2009Q4,79.370,298.064,111.544,496.904,NaN,11.200,364.809,A,423860.0
2,1004,20100831,2010,1.0,INDL,C,D,STD,AIR,000361105,...,2010Q1,52.155,334.392,NaN,520.001,NaN,13.674,404.393,A,423860.0
3,1004,20101130,2010,2.0,INDL,C,D,STD,AIR,000361105,...,2010Q2,49.320,379.795,NaN,524.548,NaN,16.814,454.858,A,423860.0
4,1004,20110228,2010,3.0,INDL,C,D,STD,AIR,000361105,...,2010Q3,54.716,379.242,NaN,523.464,NaN,17.918,458.035,A,423860.0


In [13]:
file = "../../data/companyData/compustatAll.csv"
compustatAll = pd.read_csv(file, encoding = 'unicode_escape')

Seems like we've forgotten the naics code here.

In [14]:
compustatAll.columns

Index(['gvkey', 'datadate', 'fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc',
       'datafmt', 'tic', 'cusip', 'conm', 'curcdq', 'curncdq', 'currtrq',
       'datacqtr', 'datafqtr', 'cogsq', 'invtq', 'niq', 'revtq', 'saleq',
       'cik', 'costat'],
      dtype='object')

In [7]:
'''compustat = compustat[(compustat.fyearq > 2009) & (compustat.fyearq < 2020)].reset_index()[['fyearq', 'fqtr', 'gvkey', 'conm', 'tic', 'naics','curcdq',
           'niq', 'revtq', 'cogsq', 'invtq']]'''

compustat = compustat[['fyearq', 'fqtr', 'gvkey', 'conm', 'tic','curcdq',
           'niq', 'revtq', 'cogsq', 'invtq','saleq']]
compustat.head()

,fyearq,fqtr,gvkey,conm,tic,curcdq,niq,revtq,cogsq,invtq,saleq
0,1975,1.0,1000,A & E PLASTIK PAK INC,AE.2,USD,0.190,11.295,NaN,NaN,11.295
1,1975,2.0,1000,A & E PLASTIK PAK INC,AE.2,USD,0.875,14.441,NaN,NaN,14.441
2,1975,3.0,1000,A & E PLASTIK PAK INC,AE.2,USD,0.613,13.122,NaN,NaN,13.122
3,1975,4.0,1000,A & E PLASTIK PAK INC,AE.2,USD,0.606,12.334,35.237,6.227,12.334
4,1976,1.0,1000,A & E PLASTIK PAK INC,AE.2,USD,0.726,16.226,11.138,6.369,16.226


In [ ]:
compustat.rename(columns = {'fyearq':'year',
                            'fqtr':  'qtr',
                            'chq':   'cash', 
                            'conm':  'companyName',
                            'niq':   'netIncome',
                            'revtq': 'totalRevenue',
                            'cogsq': 'costGoodsSold',
                            'invtq': 'totalInv',
                            'saleq': 'sales'
                           },
                 inplace = True)

In [ ]:
compustat.head()

In [ ]:
compustat.curcdq.value_counts()

In [ ]:
compustatNext = compustat.copy()

compustatNext['year'] -= 1
compustatNext.rename(columns = {'sales':'salesNext',
                                'netIncome':'netIncomeNext',
                                'totalRevenue':'totalRevenueNext',
                                'costGoodsSold':'costGoodsSoldNext',
                                'totalInv':'totalInvNext'
                           },
                 inplace = True)

In [ ]:
compustatNext.head()

In [ ]:
compustatChanges = compustat.merge(compustatNext)

In [ ]:
compustatChanges['salesChange'] = (compustatChanges.salesNext - compustatChanges.sales)/compustatChanges.sales

compustatChanges['incomeChange'] = (compustatChanges.netIncomeNext - compustatChanges.netIncome)/compustatChanges.netIncome

compustatChanges['revenueChange'] = (compustatChanges.totalRevenueNext - compustatChanges.totalRevenue)/compustatChanges.totalRevenue

compustatChanges['revenueChangeAbsolute'] = (compustatChanges.totalRevenueNext - compustatChanges.totalRevenue)

compustatChanges['costChange'] = (compustatChanges.costGoodsSoldNext - compustatChanges.costGoodsSold)/compustatChanges.costGoodsSold

compustatChanges['inventoryChange'] = (compustatChanges.totalInvNext - compustatChanges.totalInv)/compustatChanges.totalInv

compustatChanges = compustatChanges[['year', 'qtr', 'gvkey', 'companyName', 
                                     'tic', 'curcdq','salesChange','incomeChange','revenueChange',
                                     'revenueChangeAbsolute','costChange','inventoryChange']]


In [ ]:
compustatChanges.to_csv("../../data/compustatChanges_all.csv")

In [ ]:
83000*1000000/1e9

In [ ]:
compustatChanges.shape[0]*0.002*4.5*compustatChanges.revenueChangeAbsolute.mean()

In [ ]:
compustatChanges.head()

In [ ]:
compustatChanges.year

### Company-level controls

In [23]:
file = "../../data/companyData/compustatControls.csv"
compustatControls = pd.read_csv(file, encoding = 'unicode_escape')
compustatControls.head()

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,curcdq,datacqtr,datafqtr,atq,niq,costat,prccq,naics,sic
0,1000,19660331,1966,1.0,INDL,C,D,STD,AE.2,USD,1966Q1,1966Q1,NaN,NaN,I,NaN,NaN,3089.0
1,1000,19660630,1966,2.0,INDL,C,D,STD,AE.2,USD,1966Q2,1966Q2,NaN,NaN,I,NaN,NaN,3089.0
2,1000,19660930,1966,3.0,INDL,C,D,STD,AE.2,USD,1966Q3,1966Q3,NaN,NaN,I,NaN,NaN,3089.0
3,1000,19661231,1966,4.0,INDL,C,D,STD,AE.2,USD,1966Q4,1966Q4,NaN,0.164,I,NaN,NaN,3089.0
4,1000,19670331,1967,1.0,INDL,C,D,STD,AE.2,USD,1967Q1,1967Q1,NaN,NaN,I,NaN,NaN,3089.0


In [24]:
earliestYear = compustatControls.groupby('gvkey')['fyearq'].min().reset_index().\
    rename(columns = {'fyearq': 'earliestYear'})
earliestYear.head()

,gvkey,earliestYear
0,1000,1966
1,1001,1983
2,1003,1982
3,1004,1968
4,1005,1978


---------------------------------------

In [41]:
otherControls = compustatControls[['gvkey','fyearq','fqtr','sic','naics','atq','niq']]

In [44]:
otherControls['roa'] = otherControls['niq']/otherControls['atq']

otherControls.rename(columns = {'niq': 'netIncome',
                               'atq':  'assets'},inplace = True)


/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [45]:
otherControls.loc[(otherControls.sic >=100) & (otherControls.sic <= 799), 'famafrench'] = 1
otherControls.loc[(otherControls.sic == 2048), 'famafrench'] = 1

otherControls.loc[(otherControls.sic >=2000) & (otherControls.sic <= 2046), 'famafrench'] = 2
otherControls.loc[(otherControls.sic >=2050) & (otherControls.sic <= 2063), 'famafrench'] = 2
otherControls.loc[(otherControls.sic >=2070) & (otherControls.sic <= 2079), 'famafrench'] = 2
otherControls.loc[(otherControls.sic >=2090) & (otherControls.sic <= 2095), 'famafrench'] = 2
otherControls.loc[(otherControls.sic >=2098) & (otherControls.sic <= 2099), 'famafrench'] = 2

otherControls.loc[(otherControls.sic >=2064) & (otherControls.sic <= 2068), 'famafrench'] = 3
otherControls.loc[(otherControls.sic >=2086) & (otherControls.sic <= 2087), 'famafrench'] = 3
otherControls.loc[(otherControls.sic >=2096) & (otherControls.sic <= 2097), 'famafrench'] = 3

otherControls.loc[(otherControls.sic >=2080) & (otherControls.sic <= 2085), 'famafrench'] = 4

otherControls.loc[(otherControls.sic >=2100) & (otherControls.sic <= 2199), 'famafrench'] = 5

otherControls.loc[(otherControls.sic >=900)  & (otherControls.sic <= 999), 'famafrench']  = 6
otherControls.loc[(otherControls.sic >=3650) & (otherControls.sic<= 3652), 'famafrench']  = 6
otherControls.loc[(otherControls.sic ==3732), 'famafrench']  = 6
otherControls.loc[(otherControls.sic >=3930)  & (otherControls.sic<= 3949), 'famafrench']  = 6

otherControls.loc[(otherControls.sic >=7800)  & (otherControls.sic<= 7841), 'famafrench']  = 7
otherControls.loc[(otherControls.sic >=7900)  & (otherControls.sic<= 7999), 'famafrench']  = 7

otherControls.loc[(otherControls.sic >=2700)  & (otherControls.sic<= 2749), 'famafrench']  = 8
otherControls.loc[(otherControls.sic >=2770)  & (otherControls.sic<= 2799), 'famafrench']  = 8

otherControls.loc[(otherControls.sic ==2047), 'famafrench']  = 9
otherControls.loc[(otherControls.sic >=2391)  & (otherControls.sic<= 2392), 'famafrench']  = 9
otherControls.loc[(otherControls.sic >=2510)  & (otherControls.sic<= 2519), 'famafrench']  = 9
otherControls.loc[(otherControls.sic >=2590)  & (otherControls.sic<= 2599), 'famafrench']  = 9
otherControls.loc[(otherControls.sic >=2840)  & (otherControls.sic<= 2844), 'famafrench']  = 9
otherControls.loc[(otherControls.sic >=3160)  & (otherControls.sic<= 3199), 'famafrench']  = 9
otherControls.loc[(otherControls.sic >=3229)  & (otherControls.sic<= 3231), 'famafrench']  = 9
otherControls.loc[(otherControls.sic ==3260), 'famafrench']  = 9
otherControls.loc[(otherControls.sic >=3262)  & (otherControls.sic<= 3263), 'famafrench']  = 9
otherControls.loc[(otherControls.sic ==3269), 'famafrench']  = 9
otherControls.loc[(otherControls.sic >=3630)  & (otherControls.sic<= 3639), 'famafrench']  = 9
otherControls.loc[(otherControls.sic >=3750)  & (otherControls.sic<= 3751), 'famafrench']  = 9
otherControls.loc[(otherControls.sic ==3800), 'famafrench']  = 9
otherControls.loc[(otherControls.sic >=3229)  & (otherControls.sic<= 3231), 'famafrench']  = 9

otherControls.loc[(otherControls.sic >=3229)  & (otherControls.sic<= 3231), 'famafrench']  = 9


otherControls.loc[(otherControls.sic>=3860) & (otherControls.sic<=3879), 'famafrench']  = 9
otherControls.loc[(otherControls.sic>=3910) & (otherControls.sic<=3919), 'famafrench']  = 9
otherControls.loc[(otherControls.sic>=3960) & (otherControls.sic<=3961), 'famafrench']  = 9
otherControls.loc[(otherControls.sic==3991), 'famafrench']  = 9
otherControls.loc[(otherControls.sic==3995), 'famafrench']  = 9


otherControls.loc[(otherControls.sic>=2300) & (otherControls.sic<=2390), 'famafrench']  = 10
otherControls.loc[(otherControls.sic>=3020) & (otherControls.sic<=3021), 'famafrench']  = 10
otherControls.loc[(otherControls.sic>=3100) & (otherControls.sic<=3111), 'famafrench']  = 10
otherControls.loc[(otherControls.sic>=3130) & (otherControls.sic<=3159), 'famafrench']  = 10
otherControls.loc[(otherControls.sic==3965), 'famafrench']  = 10

otherControls.loc[(otherControls.sic>=8000) & (otherControls.sic<=8099), 'famafrench']  = 11

otherControls.loc[(otherControls.sic==3693), 'famafrench']  = 12
otherControls.loc[(otherControls.sic>=3840) & (otherControls.sic<=3851), 'famafrench']  = 12

otherControls.loc[(otherControls.sic>=2830) & (otherControls.sic<=2836), 'famafrench']  = 13

otherControls.loc[(otherControls.sic>=2800) & (otherControls.sic<=2829), 'famafrench']  = 14
otherControls.loc[(otherControls.sic>=2850) & (otherControls.sic<=2899), 'famafrench']  = 14

otherControls.loc[(otherControls.sic==3000), 'famafrench']  = 15
otherControls.loc[(otherControls.sic>=3050) & (otherControls.sic<=3099), 'famafrench']  = 15

otherControls.loc[(otherControls.sic>=2200) & (otherControls.sic<=2295), 'famafrench']  = 16
otherControls.loc[(otherControls.sic>=2297) & (otherControls.sic<=2299), 'famafrench']  = 16
otherControls.loc[(otherControls.sic>=2393) & (otherControls.sic<=2395), 'famafrench']  = 16
otherControls.loc[(otherControls.sic>=2397) & (otherControls.sic<=2399), 'famafrench']  = 16

otherControls.loc[(otherControls.sic>=800) & (otherControls.sic<=899), 'famafrench']  = 17
otherControls.loc[(otherControls.sic>=2400) & (otherControls.sic<=2439), 'famafrench']  = 17
otherControls.loc[(otherControls.sic>=2450) & (otherControls.sic<=2459), 'famafrench']  = 17
otherControls.loc[(otherControls.sic>=2490) & (otherControls.sic<=2499), 'famafrench']  = 17
otherControls.loc[(otherControls.sic>=2950) & (otherControls.sic<=2952), 'famafrench']  = 17
otherControls.loc[(otherControls.sic>=3200) & (otherControls.sic<=3219), 'famafrench']  = 17
otherControls.loc[(otherControls.sic>=3240) & (otherControls.sic<=3259), 'famafrench']  = 17
otherControls.loc[(otherControls.sic==3261), 'famafrench']  = 17
otherControls.loc[(otherControls.sic==3264), 'famafrench']  = 17
otherControls.loc[(otherControls.sic>=3270) & (otherControls.sic<=3299), 'famafrench']  = 17
otherControls.loc[(otherControls.sic>=3420) & (otherControls.sic<=3442), 'famafrench']  = 17
otherControls.loc[(otherControls.sic>=3446) & (otherControls.sic<=3452), 'famafrench']  = 17
otherControls.loc[(otherControls.sic>=3490) & (otherControls.sic<=3499), 'famafrench']  = 17
otherControls.loc[(otherControls.sic==3996), 'famafrench']  = 17

otherControls.loc[(otherControls.sic>=1500) & (otherControls.sic<=1549), 'famafrench']  = 18
otherControls.loc[(otherControls.sic>=1600) & (otherControls.sic<=1699), 'famafrench']  = 18
otherControls.loc[(otherControls.sic>=1700) & (otherControls.sic<=1799), 'famafrench']  = 18

otherControls.loc[(otherControls.sic>=3300) & (otherControls.sic<=3369), 'famafrench']  = 19
otherControls.loc[(otherControls.sic>=3390) & (otherControls.sic<=3399), 'famafrench']  = 19

otherControls.loc[(otherControls.sic==3400), 'famafrench']  = 20
otherControls.loc[(otherControls.sic>=3443) & (otherControls.sic<=3444), 'famafrench']  = 20
otherControls.loc[(otherControls.sic>=3460) & (otherControls.sic<=3479), 'famafrench']  = 20

otherControls.loc[(otherControls.sic>=3510) & (otherControls.sic<=3536), 'famafrench']  = 21
otherControls.loc[(otherControls.sic>=3540) & (otherControls.sic<=3569), 'famafrench']  = 21
otherControls.loc[(otherControls.sic>=3580) & (otherControls.sic<=3599), 'famafrench']  = 21

otherControls.loc[(otherControls.sic>=3600) & (otherControls.sic<=3621), 'famafrench']  = 22
otherControls.loc[(otherControls.sic>=3623) & (otherControls.sic<=3629), 'famafrench']  = 22
otherControls.loc[(otherControls.sic>=3640) & (otherControls.sic<=3646), 'famafrench']  = 22
otherControls.loc[(otherControls.sic>=3648) & (otherControls.sic<=3649), 'famafrench']  = 22
otherControls.loc[(otherControls.sic==3660), 'famafrench']  = 22
otherControls.loc[(otherControls.sic>=3690) & (otherControls.sic<=3692), 'famafrench']  = 22
otherControls.loc[(otherControls.sic==3699), 'famafrench']  = 22

otherControls.loc[(otherControls.sic==3900), 'famafrench']  = 23
otherControls.loc[(otherControls.sic==3990), 'famafrench']  = 23
otherControls.loc[(otherControls.sic==3999), 'famafrench']  = 23
otherControls.loc[(otherControls.sic>=9900) & (otherControls.sic<=9999), 'famafrench']  = 23

otherControls.loc[(otherControls.sic==2296), 'famafrench']  = 24
otherControls.loc[(otherControls.sic==2396), 'famafrench']  = 24
otherControls.loc[(otherControls.sic>=3011) & (otherControls.sic<=3011), 'famafrench']  = 24
otherControls.loc[(otherControls.sic==3537), 'famafrench']  = 24
otherControls.loc[(otherControls.sic==3647), 'famafrench']  = 24
otherControls.loc[(otherControls.sic==3694), 'famafrench']  = 24
otherControls.loc[(otherControls.sic>=3700) & (otherControls.sic<=3716), 'famafrench']  = 24
otherControls.loc[(otherControls.sic>=3790) & (otherControls.sic<=3792), 'famafrench']  = 24
otherControls.loc[(otherControls.sic==3799), 'famafrench']  = 24

otherControls.loc[(otherControls.sic>=3720) & (otherControls.sic<=3729), 'famafrench']  = 25

otherControls.loc[(otherControls.sic>=3730) & (otherControls.sic<=3731), 'famafrench']  = 26
otherControls.loc[(otherControls.sic>=3740) & (otherControls.sic<=3743), 'famafrench']  = 26

otherControls.loc[(otherControls.sic>=3480) & (otherControls.sic<=3489), 'famafrench']  = 27
otherControls.loc[(otherControls.sic>=3760) & (otherControls.sic<=3769), 'famafrench']  = 27
otherControls.loc[(otherControls.sic==3795), 'famafrench']  = 27

otherControls.loc[(otherControls.sic>=1040) & (otherControls.sic<=1049), 'famafrench']  = 28

otherControls.loc[(otherControls.sic>=1000) & (otherControls.sic<=1039), 'famafrench']  = 29
otherControls.loc[(otherControls.sic>=1060) & (otherControls.sic<=1099), 'famafrench']  = 29
otherControls.loc[(otherControls.sic>=1400) & (otherControls.sic<=1499), 'famafrench']  = 29

otherControls.loc[(otherControls.sic>=1200) & (otherControls.sic<=1299), 'famafrench']  = 30

otherControls.loc[(otherControls.sic>=1310) & (otherControls.sic<=1389), 'famafrench']  = 31
otherControls.loc[(otherControls.sic>=2900) & (otherControls.sic<=2911), 'famafrench']  = 31
otherControls.loc[(otherControls.sic>=2990) & (otherControls.sic<=2999), 'famafrench']  = 31

otherControls.loc[(otherControls.sic>=4900) & (otherControls.sic<=4999), 'famafrench']  = 32

otherControls.loc[(otherControls.sic>=4800) & (otherControls.sic<=4899), 'famafrench']  = 33

otherControls.loc[(otherControls.sic>=7020) & (otherControls.sic<=7021), 'famafrench']  = 34
otherControls.loc[(otherControls.sic>=7030) & (otherControls.sic<=7039), 'famafrench']  = 34
otherControls.loc[(otherControls.sic>=7200) & (otherControls.sic<=7212), 'famafrench']  = 34
otherControls.loc[(otherControls.sic>=7215) & (otherControls.sic<=7299), 'famafrench']  = 34
otherControls.loc[(otherControls.sic==7395), 'famafrench']  = 34
otherControls.loc[(otherControls.sic==7500), 'famafrench']  = 34
otherControls.loc[(otherControls.sic>=7520) & (otherControls.sic<=7549), 'famafrench']  = 34
otherControls.loc[(otherControls.sic>=7600) & (otherControls.sic<=7699), 'famafrench']  = 34
otherControls.loc[(otherControls.sic>=8100) & (otherControls.sic<=8199), 'famafrench']  = 34
otherControls.loc[(otherControls.sic>=8200) & (otherControls.sic<=8299), 'famafrench']  = 34
otherControls.loc[(otherControls.sic>=8300) & (otherControls.sic<=8399), 'famafrench']  = 34
otherControls.loc[(otherControls.sic>=8400) & (otherControls.sic<=8499), 'famafrench']  = 34
otherControls.loc[(otherControls.sic>=8600) & (otherControls.sic<=8699), 'famafrench']  = 34
otherControls.loc[(otherControls.sic>=8800) & (otherControls.sic<=8899), 'famafrench']  = 34

otherControls.loc[(otherControls.sic>=2750) & (otherControls.sic<=2759), 'famafrench']  = 35
otherControls.loc[(otherControls.sic==3993), 'famafrench']  = 35
otherControls.loc[(otherControls.sic>=7300) & (otherControls.sic<=7372), 'famafrench']  = 35
otherControls.loc[(otherControls.sic>=7374) & (otherControls.sic<=7394), 'famafrench']  = 35
otherControls.loc[(otherControls.sic==7397), 'famafrench']  = 35
otherControls.loc[(otherControls.sic==7399), 'famafrench']  = 35
otherControls.loc[(otherControls.sic>=7510) & (otherControls.sic<=7519), 'famafrench']  = 35
otherControls.loc[(otherControls.sic>=8700) & (otherControls.sic<=8748), 'famafrench']  = 35
otherControls.loc[(otherControls.sic>=8900) & (otherControls.sic<=8999), 'famafrench']  = 35

otherControls.loc[(otherControls.sic>=3570) & (otherControls.sic<=3579), 'famafrench']  = 36
otherControls.loc[(otherControls.sic>=3680) & (otherControls.sic<=3689), 'famafrench']  = 36
otherControls.loc[(otherControls.sic==3695), 'famafrench']  = 36
otherControls.loc[(otherControls.sic==7373), 'famafrench']  = 36

otherControls.loc[(otherControls.sic==3622), 'famafrench']  = 37
otherControls.loc[(otherControls.sic>=3661) & (otherControls.sic<=3679), 'famafrench']  = 37
otherControls.loc[(otherControls.sic==3810), 'famafrench']  = 37
otherControls.loc[(otherControls.sic==3812), 'famafrench']  = 37

otherControls.loc[(otherControls.sic==3811), 'famafrench']  = 38
otherControls.loc[(otherControls.sic>=3820) & (otherControls.sic<=3830), 'famafrench']  = 38


otherControls.loc[(otherControls.sic>=2520) & (otherControls.sic<=2549), 'famafrench']  = 39
otherControls.loc[(otherControls.sic>=2600) & (otherControls.sic<=2639), 'famafrench']  = 39
otherControls.loc[(otherControls.sic>=2670) & (otherControls.sic<=2699), 'famafrench']  = 39
otherControls.loc[(otherControls.sic>=2760) & (otherControls.sic<=2761), 'famafrench']  = 39
otherControls.loc[(otherControls.sic>=3950) & (otherControls.sic<=3955), 'famafrench']  = 39

otherControls.loc[(otherControls.sic>=2440) & (otherControls.sic<=2449), 'famafrench']  = 40
otherControls.loc[(otherControls.sic>=2640) & (otherControls.sic<=2659), 'famafrench']  = 40
otherControls.loc[(otherControls.sic>=3210) & (otherControls.sic<=3221), 'famafrench']  = 40
otherControls.loc[(otherControls.sic>=3410) & (otherControls.sic<=3412), 'famafrench']  = 40

otherControls.loc[(otherControls.sic>=4000) & (otherControls.sic<=4099), 'famafrench']  = 41
otherControls.loc[(otherControls.sic>=4100) & (otherControls.sic<=4199), 'famafrench']  = 41
otherControls.loc[(otherControls.sic>=4200) & (otherControls.sic<=4299), 'famafrench']  = 41
otherControls.loc[(otherControls.sic>=4400) & (otherControls.sic<=4499), 'famafrench']  = 41
otherControls.loc[(otherControls.sic>=4500) & (otherControls.sic<=4599), 'famafrench']  = 41
otherControls.loc[(otherControls.sic>=4600) & (otherControls.sic<=4699), 'famafrench']  = 41
otherControls.loc[(otherControls.sic>=4700) & (otherControls.sic<=4799), 'famafrench']  = 41

otherControls.loc[(otherControls.sic>=5000) & (otherControls.sic<=5099), 'famafrench']  = 42
otherControls.loc[(otherControls.sic>=5100) & (otherControls.sic<=5199), 'famafrench']  = 42

otherControls.loc[(otherControls.sic>=5200) & (otherControls.sic<=5299), 'famafrench']  = 43
otherControls.loc[(otherControls.sic>=5300) & (otherControls.sic<=5399), 'famafrench']  = 43
otherControls.loc[(otherControls.sic>=5400) & (otherControls.sic<=5499), 'famafrench']  = 43
otherControls.loc[(otherControls.sic>=5500) & (otherControls.sic<=5599), 'famafrench']  = 43
otherControls.loc[(otherControls.sic>=5600) & (otherControls.sic<=5699), 'famafrench']  = 43
otherControls.loc[(otherControls.sic>=5700) & (otherControls.sic<=5736), 'famafrench']  = 43

otherControls.loc[(otherControls.sic>=5900) & (otherControls.sic<=5999), 'famafrench']  = 44

otherControls.loc[(otherControls.sic>=5800) & (otherControls.sic<=5813), 'famafrench']  = 44
otherControls.loc[(otherControls.sic==5890), 'famafrench']  = 44
otherControls.loc[(otherControls.sic>=7000) & (otherControls.sic<=7019), 'famafrench']  = 44
otherControls.loc[(otherControls.sic>=7040) & (otherControls.sic<=7049), 'famafrench']  = 44
otherControls.loc[(otherControls.sic==7213), 'famafrench']  = 44

otherControls.loc[(otherControls.sic>=6000) & (otherControls.sic<=6099), 'famafrench']  = 45
otherControls.loc[(otherControls.sic>=6100) & (otherControls.sic<=6199), 'famafrench']  = 45

otherControls.loc[(otherControls.sic>=6300) & (otherControls.sic<=6399), 'famafrench']  = 46
otherControls.loc[(otherControls.sic>=6400) & (otherControls.sic<=6411), 'famafrench']  = 46

otherControls.loc[(otherControls.sic>=6500) & (otherControls.sic<=6553), 'famafrench']  = 47

otherControls.loc[(otherControls.sic>=6200) & (otherControls.sic<=6299), 'famafrench']  = 48
otherControls.loc[(otherControls.sic>=6700) & (otherControls.sic<=6799), 'famafrench']  = 48

print(otherControls.head())

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


   gvkey  fyearq  fqtr     sic  naics  assets  netIncome  roa  famafrench
0   1000    1966   1.0  3089.0    NaN     NaN        NaN  NaN        15.0
1   1000    1966   2.0  3089.0    NaN     NaN        NaN  NaN        15.0
2   1000    1966   3.0  3089.0    NaN     NaN        NaN  NaN        15.0
3   1000    1966   4.0  3089.0    NaN     NaN      0.164  NaN        15.0
4   1000    1967   1.0  3089.0    NaN     NaN        NaN  NaN        15.0


In [50]:
otherControls = otherControls[['gvkey','fyearq','fqtr','netIncome','roa','famafrench']]
otherControls['year_toMatchOn'] = otherControls['fyearq'] + 3 

otherControls = otherControls.merge(earliestYear)

otherControls.head()

,gvkey,fyearq,fqtr,netIncome,roa,famafrench,year_toMatchOn,earliestYear
0,1000,1966,1.0,NaN,NaN,15.0,1969,1966
1,1000,1966,2.0,NaN,NaN,15.0,1969,1966
2,1000,1966,3.0,NaN,NaN,15.0,1969,1966
3,1000,1966,4.0,0.164,NaN,15.0,1969,1966
4,1000,1967,1.0,NaN,NaN,15.0,1970,1966


In [52]:
otherControls.earliestYear.max()

2021

In [54]:
otherControls.to_csv('../../data/companyData/otherControls.csv')

# Location Data
This is not our final source of data. We use it as a complementary source to match the companies, as many of the ompanies have similar address names in IG and CStat, especially at the end of the data (the last address in IG should match with the CStat address, as the cstat addresses are not updated).

In [ ]:
file = "../../data/cstatLocations.csv"
locations = pd.read_csv(file)[['fyear','gvkey','addzip','state']]


locations = locations[~locations.fyear.isna()]
locations = locations[~locations.addzip.isna()]
print(locations.shape)

locations.rename(columns = {'fyear': 'year'},inplace = True)
locations.reset_index(inplace = True,drop= True)
locations['year'] = locations['year'].astype('int64')
locations.drop_duplicates(inplace = True)

In [ ]:
locations.shape

In [ ]:
compustatChanges.head()

In [ ]:
companyHQ = compustatChanges.merge(locations,how='left')

In [ ]:
companyHQ = companyHQ[~companyHQ.addzip.isna()]

In [ ]:
sum(companyHQ.addzip.isna())

Merge the zip code information with the change information.

In [ ]:
zipFile = "../../data/zipLatLong.csv"
zips = pd.read_csv(zipFile)
'''[['ZIP','LAT','LNG']].rename(columns = {'ZIP': 'addzip'})
zips['addzip'] = zips.addzip.astype('object')'''

zips = zips['ZIP,LAT,LNG'].str.split(',', expand=True)

zips.columns = ['addzip','latitude','longitude']

print(zips.head())

In [ ]:
companyHQZips = companyHQ.merge(zips)
print(companyHQZips.head(),companyHQZips.shape)

In [ ]:
companyHQZips = companyHQZips.drop_duplicates()

In [ ]:
companyHQZips.to_csv("../../data/companyHQZips.csv")

-------------------

# Segments Data

In [ ]:
import datetime

In [ ]:
file = "../../data/companyData/compustatSegments.csv"
segments = pd.read_csv(file)

In [ ]:
segments['srcdate'] = pd.DatetimeIndex(pd.to_datetime(segments['srcdate'], format='%Y%m%d'))
segments['year']    = pd.DatetimeIndex(pd.to_datetime(segments['srcdate'], format='%Y%m%d')).year

In [ ]:
segments.head()
relats = segments[segments.ctype == 'COMPANY']
relats['customerCleaned'] = relats.cnms.str.lower()
print(relats.shape)

relats = relats[relats.customerCleaned != 'not reported'] 
print(relats.shape)

relats = relats[~relats['customerCleaned'].str.contains('customers',na = True)] 
print(relats.shape)

relats = relats[['year','gvkey','conm','customerCleaned','salecs']].drop_duplicates()

There is a linked version of Compustat that has company names merged in.

It seems like it has maybe an incomplete representation of the companies in it, but it's not too too bad of the company-company sales here.

In [ ]:
relats.head()

In [ ]:
file = "../../data/companyData/compustatSCLinked.csv"
linked = pd.read_csv(file)

linked['year'] = pd.DatetimeIndex(pd.to_datetime(linked['srcdate'], format='%Y%m%d')).year

In [ ]:
linked.salecs.sum()/relats.salecs.sum()

# [['cnms','srcdate','conm']].drop_duplicates().shape

In [ ]:
linked.head()

In [ ]:
customers = linked[['year','cgvkey']].drop_duplicatzipes().rename(columns = {'cgvkey': 'gvkey'})
suppliers = linked[['year', 'gvkey']].drop_duplicates()

In [ ]:
len(linked[['gvkey']].drop_duplicates())

In [ ]:
def gatherData(df): 
    df['yearPlus1']  = df['year'] + 1
    df['yearPlus2']  = df['year'] + 2
    df['yearPlus3']  = df['year'] + 3

    df['yearMinus1'] = df['year'] - 1
    df['yearMinus2'] = df['year'] - 2
    df['yearMinus3'] = df['year'] - 3
    
    dfMelted = pd.melt(df,id_vars=['year','gvkey'], var_name='variation', value_name='vyear').\
        drop(['variation','year'], axis = 1).rename(columns = {'vyear': 'year'}).drop_duplicates()

    return(dfMelted)

In [ ]:
suppDF = gatherData(suppliers)

In [ ]:
custDF = gatherData(customers)

In [ ]:
print(suppDF.head())
print(suppDF.shape)

print(custDF.head())
print(custDF.shape)

In [ ]:
allCompanies = suppDF.append(custDF).drop_duplicates()

In [ ]:
allCompanies.head()

In [ ]:
allCompanies.to_csv('../../data/companyData/allCompanyKeys.csv')

## Merge in some of the data

Now let's try merging in the other dataframes.

In [ ]:
def mergeCompChanges(test):
    test = test.merge(compustatChanges, how = 'left')

    test.replace([np.inf, -np.inf], np.nan, inplace=True)

    test = test.dropna(subset = ['salesChange','incomeChange','revenueChange','costChange','inventoryChange'])

    return(test)

In [ ]:
suppChanges = mergeCompChanges(suppDF)
custChanges = mergeCompChanges(custDF)

In [ ]:
custChanges.salesChange.describe()

We also need to merge in the Infogroup data here.

# Clean the company information up

In [ ]:
def cleanText(text):
    text = text.\
    replace(" CORP","").replace(" CO","").replace(" INC","").\
    replace(" LTD","").replace(" -CL A","").\
    replace(" -LP","").replace(" LP","").\
    replace("-OLD","").replace(" LLC","").\
    replace(" -CL B","").replace(" -CL i","").replace(" -CL","").\
    replace("-REDH","").replace(" CP","").\
    replace("-ADR","").replace(" PLC","").lower()
    
    
    return text

### Merge in Infogroup data
Let's use the gvkey as the unit of analysis.

In [ ]:
hq = pd.read_csv('../../data/companyData/hqPublicAll.csv').drop(["Unnamed: 0"], axis = 1).\
    rename(columns = {'archive_version_year': 'year',
                     'company': 'companyName',
                     'ticker': 'tic'})

# 'ticker': 'tic',

hq['year'] = hq.year.astype('int64')

hq.head()

In [ ]:
hq['companyNameClean'] = hq.companyName.apply(cleanText).apply(cleanText).apply(cleanText)

In [ ]:
hq.companyNameClean.head()

In [ ]:
ig_unique = hq[['tic','companyNameClean']].drop_duplicates()
ig_unique.shape

In [ ]:
compIDs = suppChanges.append(custChanges)[['companyName','tic','gvkey']].drop_duplicates().reset_index().\
    drop(['index'],axis = 1)
compIDs.shape

In [ ]:
compIDs['companyNameClean'] = compIDs.companyName.apply(cleanText)

Look for compustat IDs that match infogroup IDs. First off, look to see if company name or tic is in infogroup names or tickers.

In [ ]:
compIDs.head()

In [ ]:
compIDs['hasMatch'] = (compIDs.companyNameClean.isin(ig_unique.companyNameClean) | compIDs.tic.isin(ig_unique.tic))
compIDsNeedsMatch   = compIDs[~compIDs.hasMatch]
compIDsNeedsMatch.shape

In [ ]:
compIDs.companyNameClean

In [ ]:
def closestName(text):
    matchOrder = process.extract(text, ig_unique.companyNameClean, 
                               scorer=fuzz.token_sort_ratio)
    return(matchOrder)


In [ ]:
start = time.time()
closestNames = compIDsNeedsMatch.companyNameClean[0:50].apply(closestName)

time.time() - start

In [ ]:
compIDsNeedsMatch.companyNameClean[0:50]

In [ ]:
closestNames

In [ ]:
compIDsNeedsMatch.to_csv('../../data/companyData/matchesNeeded.csv')

In [ ]:
closestNames.to_csv('../../data/companyData/closestNames.csv')

# Stock price data

In [ ]:
file = "data/crsp_raw.csv"
raw = pd.read_csv(file, encoding = 'unicode_escape')

In [ ]:
raw.head()

Let's follow Evan's crsp cleaning and linking process.

1. Drop firms not traded on NYSE or NASDAQ.

In [ ]:
other_exch = crsp[ (raw['PRIMEXCH'] != 'Q') & (raw['PRIMEXCH'] != 'N') ].index
raw.drop(other_exch, inplace=True)
raw.shape

2. Drop entries with missing return or volume information.

In [ ]:
raw.dropna(subset=['VOL', 'RET','date'],inplace=True)
raw['VOL'] = raw['VOL'].astype('int')
raw.drop(raw[raw['RET']=='C'].index,inplace=True)
raw['RET'] = raw['RET'].astype('float')
raw['date'] = raw['date'].astype('int')

3. Drop tickers with average daily trading volume less than 100K shares.

In [ ]:
adv = raw[['VOL','TICKER']].groupby(['TICKER']).mean()
low_vol = adv.drop(adv[adv['VOL']>=100000].index)
low_vol_index = raw[raw['TICKER'].isin(low_vol.index)].index
raw.drop(low_vol_index,inplace=True)
raw.shape

4. Drop tickers that traded for less than 20% of the period

In [ ]:
ndays = len(raw.date.unique())
obs = raw['TICKER'].value_counts()
rare_obs = obs[obs<0.2*ndays]
rare_index = raw[raw['TICKER'].isin(rare_obs.index)].index
raw.drop(rare_index,inplace=True)
raw.shape

In [ ]:
raw = raw.drop_duplicates()

In [ ]:
raw.shape

Merge in GVKEY information for linking with compustat.

gv_cik will come in hand with another merge maybe; but to get to compustat, looks like we just need gvkey - permco.

Some of the keys seem to be used twice. Let's see if they're re-used when a company has gone out of business.

In [ ]:
# gv_cik = pd.read_csv("data/cik_gvkey.csv",dtype={'cik':str})[['cik','gvkey']]
gv_permco = pd.read_csv("data/cik_permco.csv",dtype={'cik':str})
# [['gvkey','LPERMCO']].rename(columns = {'LPERMCO': 'PERMCO'}).drop_duplicates()

gv_permco.loc[gv_permco.LINKENDDT == 'E', 'LINKENDDT'] = '20210101'
gv_permco['existsCurrently'] = pd.DatetimeIndex(pd.to_datetime(gv_permco.LINKENDDT.astype(str), format='%Y%m%d')).year

gv_permco = gv_permco[gv_permco.existsCurrently > 2010]
print(gv_permco.shape)
print(gv_permco.head())

gv_permco = gv_permco[['gvkey','LPERMCO','conm']].rename(columns = {'LPERMCO': 'PERMCO'}).drop_duplicates()

In [ ]:
for company in gv_permco[gv_permco.PERMCO.duplicated()].conm:
    print(company)

Drop the duplicated entries. We may want to revisit this later on.

In [ ]:
gv_permco = gv_permco[~gv_permco.PERMCO.isin(list(gv_permco.PERMCO[gv_permco.PERMCO.duplicated()]))]

Now just merge the raw return information with this company information so we can link everything.

In [ ]:
rawMergeable = pd.merge(raw,gv_permco,on='PERMCO',how = 'left')
rawMergeable.shape

In [ ]:
raw.head()

In [ ]:
rawMergeable = rawMergeable.drop(['RETX'],axis = 1)
rawMergeable['date'] = pd.to_datetime(rawMergeable['date'].astype(str), format='%Y%m%d')
rawMergeable['year'] = pd.DatetimeIndex(rawMergeable['date']).year
rawMergeable.head()

In [ ]:
outfile =  'data/stockReturns.pkl'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(rawMergeable, pickle_file)

----------------------------------------------

In [ ]:
crsp2010s = crsp[['date','year','PERMNO','COMNAM','PRC']]

In [ ]:
sum(crsp.RET == crsp.RETX)